## `validate_geojson_features('/path/to/unvalidated_geojson', '/path/to/invalid_features', '/path/to/valid_features')` 

1. Validate each feature within a GeoJSON file.
2. Separate the invalid features into a new GeoJSON file `{filename}_invalid_features.geojson`.
3. Save the remaining valid features into a new GeoJSON file `{filename}_validated.geojson`.
4. Ensure all outputs are properly formed GeoJSON files in WGS84 format, even if they contain empty geometries or invalid features.
5. Create the specified directories if they don't already exist, and overwrite previous outputs.

6. **Directory Setup**:
   - The function ensures that the `invalid_features_path` and `valid_features_path` directories exist, creating them if they don't.

7. **Processing Each GeoJSON File**:
   - The function iterates through all files in the `unvalidated_geojson_path`.
   - It reads the content of each GeoJSON file using the `geojson` library.

8. **Feature Validation**:
   - For each feature in the GeoJSON, the function checks the coordinate system and converts it to WGS84 if necessary using the `pyproj` library.
   - It then validates the feature's structure using the `validate_structure` function from the `geojson-validator` module.
   - Valid features are collected into a list of valid features, and invalid ones into a list of invalid features.

9. **Writing Output Files**:
   - The function writes the valid features to a file named `{filename}_validated.geojson` in the `valid_features_path` directory.
   - The invalid features are written to a file named `{filename}_invalid_features.geojson` in the `invalid_features_path` directory.

10. **Example Usage**:
   - To use this function, replace the directory paths in the example usage section with your actual directory paths.

### Requirements:
- The `geojson`, `geojson-validator`, and `pyproj` packages need to be installed.
- Ensure the input files are valid GeoJSON structures at the top level (even if individual features may be invalid).

In [ ]:
import os
import json
import geojson
from geojson_validator import validate_structure
from pyproj import Transformer

def validate_geojson_features(unvalidated_geojson_path, invalid_features_path, valid_features_path):
    # Create directories if they don't exist
    os.makedirs(invalid_features_path, exist_ok=True)
    os.makedirs(valid_features_path, exist_ok=True)

    # Iterate through each file in the unvalidated_geojson_path
    for filename in os.listdir(unvalidated_geojson_path):
        file_path = os.path.join(unvalidated_geojson_path, filename)

        # Skip if it's not a file
        if not os.path.isfile(file_path):
            continue

        # Read the GeoJSON file
        with open(file_path, 'r') as geojson_file:
            data = geojson.load(geojson_file)

        # Initialize lists for valid and invalid features
        valid_features = []
        invalid_features = []

        # Validate each feature
        for feature in data.get('features', []):
            # Convert to WGS84 if necessary
            if feature.get('geometry', {}).get('crs') and feature['geometry']['crs']['properties']['name'] != 'EPSG:4326':
                transformer = Transformer.from_crs(feature['geometry']['crs']['properties']['name'], 'EPSG:4326', always_xy=True)
                coordinates = feature['geometry']['coordinates']
                new_coordinates = transformer.transform(*coordinates[::-1])[::-1]
                feature['geometry']['coordinates'] = new_coordinates
                feature['geometry']['crs'] = {"type": "name", "properties": {"name": "EPSG:4326"}}

            # Validate the structure of the feature
            is_valid, error_message = validate_structure(json.dumps(feature))

            if is_valid:
                valid_features.append(feature)
            else:
                invalid_features.append(feature)

        # Prepare output GeoJSON for valid and invalid features
        valid_geojson = geojson.FeatureCollection(valid_features)
        invalid_geojson = geojson.FeatureCollection(invalid_features)

        # Define output file paths
        valid_output_file = os.path.join(valid_features_path, f"{os.path.splitext(filename)[0]}_validated.geojson")
        invalid_output_file = os.path.join(invalid_features_path, f"{os.path.splitext(filename)[0]}_invalid_features.geojson")

        # Write the valid features to a new GeoJSON file
        with open(valid_output_file, 'w') as valid_file:
            geojson.dump(valid_geojson, valid_file)

        # Write the invalid features to a new GeoJSON file
        with open(invalid_output_file, 'w') as invalid_file:
            geojson.dump(invalid_geojson, invalid_file)

# Example usage:
# validate_geojson_features('/path/to/unvalidated_geojson', '/path/to/invalid_features', '/path/to/valid_features')
